<a href="https://colab.research.google.com/github/dhanushnayak/BERT/blob/main/BERT_SPAM_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.1 MB/s 
     |████████████████████████████████| 596 kB 55.6 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
     |████████████████████████████████| 6.6 MB 70.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("/content/spam.csv",encoding="ISO-8859-1").iloc[:,:2]

In [5]:
from sklearn.model_selection import train_test_split

x = df.iloc[:,1].to_list()
y = df.iloc[:,0].map({"spam":1,"ham":0}).to_list()

In [6]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [7]:
from transformers import BertTokenizer,BertForSequenceClassification,TrainingArguments,Trainer

In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
train_encoding = tokenizer(x_train,padding=True)
test_encoding = tokenizer(x_test,padding=True)

In [26]:
train_encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [10]:
import torch
from torch.utils.data import TensorDataset,Dataset
class CreateDataset(Dataset):
  def __init__(self,encoding,labels):
    self.encoding = encoding
    self.labels = labels
  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encoding.items()}
    item['labels']=torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

trainset = CreateDataset(train_encoding,y_train)

testset = CreateDataset(test_encoding,y_test)

In [30]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [41]:
args = TrainingArguments(
  output_dir="/content/outmodel", 
  overwrite_output_dir=True,
  logging_dir='/logs',
  num_train_epochs=1,
  logging_steps=10,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=64,
  weight_decay=0.1,
  learning_rate=1e-5
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=trainset,
    eval_dataset= testset
)

In [43]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3900
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 244


Step,Training Loss
10,0.050900
20,0.042800
30,0.024800
40,0.021900
50,0.013900
60,0.003200
70,0.024900
80,0.002600
90,0.002900
100,0.001700




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=244, training_loss=0.025517721967695313, metrics={'train_runtime': 157.6202, 'train_samples_per_second': 24.743, 'train_steps_per_second': 1.548, 'total_flos': 442920739482000.0, 'train_loss': 0.025517721967695313, 'epoch': 1.0})

In [49]:
trainer.save_model("/content/modelr")

Saving model checkpoint to /content/modelr
Configuration saved in /content/modelr/config.json
Model weights saved in /content/modelr/pytorch_model.bin


In [44]:
from transformers import pipeline

In [50]:
model_r = pipeline("text-classification",model="/content/modelr",tokenizer=tokenizer)

loading configuration file /content/modelr/config.json
Model config BertConfig {
  "_name_or_path": "/content/modelr",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file /content/modelr/config.json
Model config BertConfig {
  "_name_or_path": "/content/modelr",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_p

In [57]:
model_r(x_test[9])

[{'label': 'LABEL_1', 'score': 0.9980745315551758}]

In [59]:
y_test[9],x_test[9]


(1,
 'REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode')

In [14]:
from torch.utils.data import Dataset
from transformers import AdamW

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

optim = AdamW(model.parameters(),lr=5e-5)
train_loader = DataLoader(trainset,batch_size=16,shuffle=True)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [19]:
num_epoch = 5
for _ in range(num_epoch):
  for train_batch in train_loader:
    optim.zero_grad()
    input_ids = train_batch['input_ids'].to(device)
    attention_mask = train_batch['attention_mask'].to(device)
    labels = train_batch['labels'].to(device)

    out = model(input_ids,attention_mask=attention_mask,labels=labels)
    print(_,out)
    loss = out[0]
    loss.backward()
    optim.step()



Streaming output truncated to the last 5000 lines.
3 SequenceClassifierOutput(loss=tensor(0.2893, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 2.8651, -3.2679],
        [-0.3678, -0.3180],
        [-3.1621,  3.3722],
        [ 3.3273, -3.7436],
        [ 3.1828, -3.6186],
        [ 3.0621, -3.4511],
        [-3.1618,  3.6586],
        [ 3.0714, -3.4998],
        [ 3.1687, -3.6821],
        [-3.2539,  3.6640],
        [ 3.0199, -3.5165],
        [-1.8690,  2.0015],
        [ 3.0535, -3.5896],
        [-3.3263,  3.6447],
        [ 3.1001, -3.5307],
        [ 2.8543, -3.3358]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
3 SequenceClassifierOutput(loss=tensor(0.0017, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 3.0234, -3.4804],
        [ 2.8081, -3.2423],
        [ 3.1370, -3.7092],
        [ 2.3649, -3.1364],
        [ 3.1446, -3.7445],
        [ 2.6954, -3.0616],
        [ 3.1271, -3.6533],
        [ 3.2441, -3.83